In [ ]:
## WDPA and WD-OECM Global Temporal Coverage Change (2000-2024)

# Import necessary Libraries
from dateutil.relativedelta import relativedelta
import datetime
from arcgis.gis import GIS
import arcpy
import math
import os
import time
import pandas as pd
from arcgis.features import FeatureLayer
from arcgis.features import FeatureLayerCollection

#Itentify the month and year corresponding to the version of the WDPA and WD-OECM used in the Protected Planet Report 2024 (August 2024)
date = datetime.datetime(2024, 8, 1)
month = date.strftime("%b")  # "Aug"
Month = date.strftime("%B")   # "August"
year = date.strftime("%Y")    # "2024"

In [ ]:
#Set default file geodatabase for intermediate outputs
arcpy.env.workspace = r"path_to_file_geodatabase_for_intermediate_outputs"

#Define paths to WDPA and WDOECM points and polygons 

# Public WDPA paths
wdpa_path = r'path_to_all_public_wdpa_point_polygon_file_geodatabase'
wdpa_poly_path = os.path.join(wdpa_path,'WDPA_poly_{0}{1}'.format(month,year))
wdpa_point_path = os.path.join(wdpa_path,'WDPA_point_{0}{1}'.format(month,year))
print(wdpa_path)

# Public WDOECM paths
wdoecm_path = r'path_to_all_public_wdoecm_point_polygon_file_geodatabase'
wdoecm_poly_path = os.path.join(wdoecm_path,'WDOECM_poly_{0}{1}'.format(month,year)) 
wdoecm_point_path = os.path.join(wdoecm_path,'WDOECM_point_{0}{1}'.format(month,year))

#Restricted WDPA paths 
restricted_wdpa_path = r'path_to_all_restricted_wdpa_point_polygon_file_geodatabase'
restricted_wdpa_poly_path = os.path.join(restricted_wdpa_path,'wdpa_poly_restricted')
restricted_wdpa_point_path = os.path.join(restricted_wdpa_path,'wdpa_point_restricted')

Datasets = ["wdpa", "wdoecm"]

#Read in and subset WDPA and WDOECM

# Public WDPA points
## Export features where REP_AREA isn't equaly to "0", DESIG_ENG is not equal to "UNESCO-MAB Biosphere Reserve", and STATUS is not equal to either "Not Reported" or "Proposed"
arcpy.conversion.ExportFeatures(wdpa_point_path,r"memory\wdpa_point_mp", "REP_AREA <> 0 And DESIG_ENG <> 'UNESCO-MAB Biosphere Reserve' And STATUS <> 'Not Reported' And STATUS <> 'Proposed'")
## Convert to single part
arcpy.management.MultipartToSinglepart("wdpa_point_mp", "wdpa_point")

# Public WDPA polygons
## Export features where DESIG_ENG is not equal to "UNESCO-MAB Biosphere Reserve", and STATUS is not equal to either "Not Reported" or "Proposed"
arcpy.conversion.ExportFeatures(wdpa_poly_path,r"memory\wdpa_poly_mp", "DESIG_ENG <> 'UNESCO-MAB Biosphere Reserve' And STATUS <> 'Not Reported' And STATUS <> 'Proposed'")
## Convert to single part
arcpy.management.MultipartToSinglepart("wdpa_poly_mp", "wdpa_poly")

# Restricted WDPA points
## Export features where REP_AREA isn't equaly to "0", DESIG_ENG is not equal to "UNESCO-MAB Biosphere Reserve", and STATUS is not equal to either "Not Reported" or "Proposed"
arcpy.conversion.ExportFeatures(restricted_wdpa_point_path,r"memory\restricted_wdpa_point_mp", "REP_AREA <> 0 And DESIG_ENG <> 'UNESCO-MAB Biosphere Reserve' And STATUS <> 'Not Reported' And STATUS <> 'Proposed'")
## Convert to single part
arcpy.management.MultipartToSinglepart("restricted_wdpa_point_mp", "restricted_wdpa_point")

# Restricted WDPA polygons
## Export features where DESIG_ENG is not equal to "UNESCO-MAB Biosphere Reserve", and STATUS is not equal to either "Not Reported" or "Proposed"
arcpy.conversion.ExportFeatures(restricted_wdpa_poly_path,r"memory\restricted_wdpa_poly_mp", "DESIG_ENG <> 'UNESCO-MAB Biosphere Reserve' And STATUS <> 'Not Reported' And STATUS <> 'Proposed'")
## Convert to single part
arcpy.management.MultipartToSinglepart("restricted_wdpa_poly_mp", "restricted_wdpa_poly")

# Public WDOECM points
## Export features where REP_AREA is not equal to "0" and STATUS is not equal to either "Not Reported" or "Proposed"
arcpy.conversion.ExportFeatures(wdoecm_point_path,r"memory\wdoecm_point_mp", "")
## Convert to single part
arcpy.management.MultipartToSinglepart("wdoecm_point_mp", "wdoecm_point")

## Export features where STATUS is not equal to either "Not Reported" or "Proposed"
arcpy.conversion.ExportFeatures(wdoecm_poly_path,r"memory\wdoecm_poly_mp", "")
## Convert to single part
arcpy.management.MultipartToSinglepart("wdoecm_poly_mp", "wdoecm_poly")

##Buffer WDPA, WDOECM, and restricted WDPA point shapefiles to REP_AREA value; converting to polygon shapefiles. 
Datasets = ["wdpa", "wdoecm", "restricted_wdpa"]

for i in Datasets:
    point_fc = f"{i}_point"
    
    # Add buffer_radius field
    arcpy.management.AddField(point_fc, "buffer_radius", "LONG", field_length=None, field_alias="", field_is_nullable="NULLABLE", field_is_required="NON_REQUIRED")
    
    # Calculate the buffer radius based on the REP_AREA field value
    arcpy.management.CalculateField(point_fc, "buffer_radius", "math.sqrt(!REP_AREA!/3.1415)", "PYTHON3", code_block="")
    
    # Create a buffer field combining the radius with a unit string
    arcpy.management.CalculateField(point_fc, "buffer_field", 'str(!buffer_radius!) + " Kilometers"', "PYTHON3", code_block="")
    
    # Perform the buffering analysis
    buffer_fc = f"{i}_point_buffered"
    arcpy.analysis.PairwiseBuffer(point_fc, buffer_fc, "buffer_field", "None", None, "GEODESIC", "0 Meters")

## Repair geometries of buffered points layers.
arcpy.management.RepairGeometry("wdpa_point_buffered", "KEEP_NULL", "OGC")
arcpy.management.RepairGeometry("wdoecm_point_buffered", "KEEP_NULL", "OGC")
arcpy.management.RepairGeometry("restricted_wdpa_point_buffered", "KEEP_NULL", "OGC")

## Merge WDPA points + polys (all), WDOECM points + polys (all), and erase protected area extent from OECM extent to produce a
# layer show additional coverage provided by OECMs only. 
arcpy.management.Merge(["wdpa_point_buffered", "wdpa_poly", "restricted_wdpa_point_buffered", "restricted_wdpa_poly"], "wdpa_merged_public_restricted")
arcpy.management.Merge(["wdoecm_point_buffered", "wdoecm_poly"], "wdoecm_merged")
arcpy.analysis.PairwiseErase("wdoecm_merged", "wdpa_merged_public_restricted", "wdoecm_merged_pa_erase", "")

## Repair geometries of newly created layers
arcpy.management.RepairGeometry("wdpa_merged_public_restricted", "KEEP_NULL", "OGC")
arcpy.management.RepairGeometry("wdoecm_merged", "KEEP_NULL", "OGC")
arcpy.management.RepairGeometry("wdoecm_merged_pa_erase", "KEEP_NULL", "OGC")

# Write intermediate WDPA and WDOECM to local file geodatabases before clearing cache to free memory. 
#WDPA
##Set working environment for output files
wdpa_temp_path = r'path_to_file_geodatabase_for_intermediate_outputs'
wdpa_temp_poly_point_path = os.path.join(wdpa_temp_path,"wdpa_merged_public_restricted")
wdpa_output_path = r'path_to_file_geodatabase_for_preprocessing_outputs'
wdpa_output_poly_point_path = os.path.join(wdpa_output_path,"wdpa_poly_buffpoint_{0}{1}".format(month, year))
arcpy.conversion.ExportFeatures(wdpa_temp_poly_point_path, wdpa_output_poly_point_path)

#WDOECM
wdoecm_temp_path = r'path_to_file_geodatabase_for_intermediate_outputs'
wdoecm_temp_poly_point_path = os.path.join(wdoecm_temp_path,"wdoecm_merged")
wdoecm_output_path = r'path_to_file_geodatabase_for_preprocessing_outputs'
wdoecm_output_poly_point_path = os.path.join(wdoecm_output_path,"wdoecm_poly_buffpoint_{0}{1}".format(month, year))
arcpy.conversion.ExportFeatures(wdoecm_temp_poly_point_path, wdoecm_output_poly_point_path)

wdoecm_pa_erase_temp_path = r'path_to_file_geodatabase_for_intermediate_outputs'
wdoecm_pa_erase_temp_poly_point_path = os.path.join(wdoecm_pa_erase_temp_path,"wdoecm_merged_pa_erase")
wdoecm_pa_erase_output_path = r'path_to_file_geodatabase_for_preprocessing_outputs'
wdoecm_pa_erase_output_poly_point_path = os.path.join(wdoecm_pa_erase_output_path,"wdoecm_pa_erase_poly_buffpoint_{0}{1}".format(month, year))
arcpy.conversion.ExportFeatures(wdoecm_pa_erase_temp_poly_point_path, wdoecm_pa_erase_output_poly_point_path)

In [ ]:
#Calculate coverage of WDPA and WDOECM, subset by differnt STATUS_YR field values.

# Define the path to the geodatabase and the feature class
basemap_path = r'path_to_file_geodatabase_containing_basemap'
basemap_name = "basemap_name"
basemap_path_name = os.path.join(basemap_path, basemap_name)
basemap = arcpy.management.MakeFeatureLayer(basemap_path_name, basemap_name)

#Define year values to subset the WDPA and WDOECM by (2000-2024) and loop through
years = ["2024","2023", "2022", "2021", "2020", 
         "2019", "2018", "2017", "2016", "2015",  "2014", "2013", "2012", "2011", "2010", 
         "2009", "2008", "2007", "2006", "2005",  "2004", "2003", "2002", "2001", "2000"]

##############################################################################################################
#####Start loop for additional processing and calculation of geoometry attributes for the WDPA (all data)####
##############################################################################################################

for i in years:
    
    start_time = time.time()
    
    #Redefine paths to WDPA preprocessing outputs.
    wdpa_output_path = r'path_to_file_geodatabase_for_preprocessing_outputs'
    wdpa_output_poly_point_path = os.path.join(wdpa_output_path,"wdpa_poly_buffpoint_{0}{1}".format(month, year))
        
    wdpa_status_yr_path = f"path_to_file_geodatabase_for_intermediate_wdpa_outputs\wdpa_statusyr{i}"
        
    #Export WDPA subset by STATUS_YR
    arcpy.conversion.ExportFeatures(
        wdpa_output_poly_point_path, 
        wdpa_status_yr_path, 
        f"STATUS_YR <= {i}")
    
    #Dice the merged WDPA subset by status year
    wdpa_dice_path = f"path_to_file_geodatabase_for_intermediate_wdpa_outputs\wdpa_statusyr{i}_dice"
    arcpy.management.Dice(
        in_features=wdpa_status_yr_path, 
        out_feature_class=wdpa_dice_path, 
        vertex_limit="5000")
    
    #repair geometry
    arcpy.management.RepairGeometry(wdpa_dice_path, "DELETE_NULL", "OGC")
            
    #Dissolve the WDPA output (single part)
    wdpa_dissolved_path = f"path_to_file_geodatabase_for_intermediate_wdpa_outputs\wdpa_statusyr{i}_dissolved"
    arcpy.analysis.PairwiseDissolve(
        in_features=wdpa_dice_path, 
        out_feature_class=wdpa_dissolved_path, 
        dissolve_field="PA_DEF",
        multi_part="SINGLE_PART")
    
    #repair geometry
    arcpy.management.RepairGeometry(wdpa_dissolved_path, "DELETE_NULL", "OGC")
        
    #Dice the merged and dissolved WDPA subset 
    wdpa_dissolve_dice_path = f"path_to_file_geodatabase_for_intermediate_wdpa_outputs\wdpa_statusyr{i}_dice2"
    arcpy.management.Dice(
        in_features=wdpa_dissolved_path, 
        out_feature_class=wdpa_dissolve_dice_path, 
        vertex_limit="5000")
        
    #repair geometry
    arcpy.management.RepairGeometry(wdpa_dissolve_dice_path, "DELETE_NULL", "OGC")
        
    #intersect WDPA with basemap
    wdpa_dissolve_dice_intersect_path = f"path_to_file_geodatabase_for_intermediate_wdpa_outputs\wdpa_statusyr{i}_dice2_intersect"
    arcpy.analysis.PairwiseIntersect(
        in_features=[wdpa_dissolve_dice_path, basemap], 
        out_feature_class=wdpa_dissolve_dice_intersect_path, 
        join_attributes="ALL", 
        output_type="INPUT")
        
    #repair geometry
    arcpy.management.RepairGeometry(wdpa_dissolve_dice_intersect_path, "DELETE_NULL", "OGC")
         
    # Calculate Geometry Attributes for WDPA
    arcpy.management.CalculateGeometryAttributes(
        wdpa_dissolve_dice_intersect_path, 
        [["PCA_AREA", "AREA_GEODESIC"]], 
        area_unit="SQUARE_KILOMETERS")
    
    # Save the final outputs
    wdpa_final_output_path = r"path_to_file_geodatabase_for_final_wdpa_outputs\WDPA_Temporal_Coverage_Aug24.gdb"
    wdpa_dissolved_output_path = os.path.join(wdpa_final_output_path, f"wdpa_{month}{year}_statusyr{i}_output")
    arcpy.management.CopyFeatures(wdpa_dissolve_dice_intersect_path, wdpa_dissolved_output_path)
    
    #Free ArcGIS Pro memory prior running next iteration.
    arcpy.management.Delete("in_memory")
    arcpy.management.Delete("memory")
         
    end_time = time.time()  # End the timer
    iteration_time = end_time - start_time  # Calculate the time taken for this iteration
    print(f"Iteration {i} took {iteration_time} seconds")
    
print("WDPA Processing Complete.")

###################################################################################################
#####Start loop for additional processing and calculation of geoometry attributes for the WDPA#####
###############(sites reported to the WDPA with reported ISO3 = "ABNJ" only)#######################
###################################################################################################

for i in years:
    
    start_time = time.time()
    
    #Redefine paths to WDPA preprocessing outputs.
    wdpa_output_path = r'path_to_file_geodatabase_for_preprocessing_outputs'
    wdpa_output_poly_point_path = os.path.join(wdpa_output_path,"wdpa_poly_buffpoint_{0}{1}".format(month, year))
    
    wdpa_status_yr_path = f"path_to_file_geodatabase_for_intermediate_wdpa_abnj_outputs\wdpa_abnj_statusyr{i}"
        
    #Export WDPA subset by STATUS_YR and ISO3 = "ABNJ"
    arcpy.conversion.ExportFeatures(
        wdpa_output_poly_point_path, 
        wdpa_status_yr_path, 
        f"STATUS_YR <= {i} AND ISO3 = 'ABNJ'")
    
    #Dice the merged WDPA subset by status year
    wdpa_dice_path = f"path_to_file_geodatabase_for_intermediate_wdpa_abnj_outputs\wdpa_abnj_statusyr{i}_dice"
    arcpy.management.Dice(
        in_features=wdpa_status_yr_path, 
        out_feature_class=wdpa_dice_path, 
        vertex_limit="10000")
    
    #Repair geometry
    arcpy.management.RepairGeometry(wdpa_dice_path, "DELETE_NULL", "OGC")
        
    #Dissolve the WDPA output (single part)
    wdpa_dissolved_path = f"path_to_file_geodatabase_for_intermediate_wdpa_abnj_outputs\wdpa_abnj_statusyr{i}_dissolved"
    arcpy.analysis.PairwiseDissolve(
        in_features=wdpa_dice_path, 
        out_feature_class=wdpa_dissolved_path, 
        dissolve_field="",
        multi_part="SINGLE_PART")
        
    #Repair geometry
    arcpy.management.RepairGeometry(wdpa_dissolved_path, "DELETE_NULL", "OGC")
        
    #Dice the merged and dissolved WDPA subset 
    wdpa_dissolve_dice_ath = f"path_to_file_geodatabase_for_intermediate_wdpa_abnj_outputs\wdpa_abnj_statusyr{i}_dice2"
    arcpy.management.Dice(
        in_features=wdpa_dissolved_path, 
        out_feature_class=wdpa_dissolve_dice_path, 
        vertex_limit="10000")
  
    #Repair geometry
    arcpy.management.RepairGeometry(wdpa_dissolve_dice_path, "DELETE_NULL", "OGC")
       
    #Intersect WDPA with basemap
    wdpa_dissolve_dice_intersect_path = f"path_to_file_geodatabase_for_intermediate_wdpa_abnj_outputs\wdpa_abnj_statusyr{i}_dice2_intersect"
    arcpy.analysis.PairwiseIntersect(
        in_features=[wdpa_dissolve_dice_path, basemap], 
        out_feature_class=wdpa_dissolve_dice_intersect_path, 
        join_attributes="ALL", 
        output_type="INPUT")
    
    #Repair geometry
    arcpy.management.RepairGeometry(wdpa_dissolve_dice_intersect_path, "DELETE_NULL", "OGC")
        
    #Calculate Geometry Attributes for WDPA
    arcpy.management.CalculateGeometryAttributes(
        wdpa_dissolve_dice_intersect_path, 
        [["PCA_AREA", "AREA_GEODESIC"]], 
        area_unit="SQUARE_KILOMETERS")
    
    #Save the final outputs
    wdpa_final_output_path = r"path_to_file_geodatabase_for_final_wdpa_outputs\WDPA_ABNJ_Temporal_Coverage_Aug24.gdb"
    wdpa_dissolved_output_path = os.path.join(wdpa_final_output_path, f"wdpa_abnj_{month}{year}_statusyr{i}_output")
    arcpy.management.CopyFeatures(wdpa_dissolve_dice_intersectpath, wdpa_dissolved_output_path)
    print(f"wdpa_status_yr_{i}_dissolved saved to {wdpa_dissolved_output_path}")
    
    #Free ArcGIS Pro memory prior running next iteration.
    arcpy.management.Delete("in_memory")
    arcpy.management.Delete("memory")
    
    end_time = time.time()  # End the timer
    iteration_time = end_time - start_time  # Calculate the time taken for this iteration
    print(f"Iteration {i} took {iteration_time} seconds")
    
print("WDPA (ABNJ only) Processing Complete.")

##############################################################################################################
#####Start loop for additional processing and calculation of geoometry attributes for the WDOECM (all data)####
##############################################################################################################

for i in years:
    
    start_time = time.time()
    
    #Redefine paths to WDOECM preprocessing outputs.
    wdoecm_pa_erase_output_path = r'path_to_file_geodatabase_for_preprocessing_outputs'
    wdoecm_pa_erase_output_poly_point_path = os.path.join(wdoecm_pa_erase_output_path,"wdoecm_pa_erase_poly_buffpoint_{0}{1}".format(month, year))
    
    wdoecm_status_yr_path = f"path_to_file_geodatabase_for_intermediate_wdoecm_outputs\wdoecm_statusyr{i}"
    
    #Export WDOECM, subset by STATUS_YR
    arcpy.conversion.ExportFeatures(
        wdoecm_pa_erase_output_poly_point_path, 
        wdoecm_status_yr_path, 
        f"STATUS_YR <= {i}")
    
    #Dice the merged WDOECM, subset by STATUS_YR
    wdoecm_dice_path = f"path_to_file_geodatabase_for_intermediate_wdoecm_outputs\wdoecm_statusyr{i}_dice"
    arcpy.management.Dice(
        in_features=wdoecm_status_yr_path, 
        out_feature_class=wdoecm_dice_path, 
        vertex_limit="10000")
    
    #Repair geometry
    arcpy.management.RepairGeometry(wdoecm_dice_path, "DELETE_NULL", "OGC")
           
    # Dissolve the WDOECM output
    wdoecm_dissolved_path = f"path_to_file_geodatabase_for_intermediate_wdoecm_outputs\wdoecm_statusyr{i}_dissolved"
    arcpy.analysis.PairwiseDissolve(
        in_features=wdoecm_dice_path, 
        out_feature_class=wdoecm_dissolved_path, 
        dissolve_field="",
        multi_part="SINGLE_PART")
    
    #Repair geometry
    arcpy.management.RepairGeometry(wdoecm_dissolved_path, "DELETE_NULL", "OGC")
    
    #Dice the merged and dissolved WDOECM, subset by STATUS_YR
    wdoecm_dissolve_dice_path = f"path_to_file_geodatabase_for_intermediate_wdoecm_outputs\wdoecm_statusyr{i}_dice2"
    arcpy.management.Dice(
        in_features=wdoecm_dissolved_path, 
        out_feature_class=wdoecm_dissolve_dice_path, 
        vertex_limit="10000")
       
    #Repair geometry
    arcpy.management.RepairGeometry(wdoecm_dissolve_dice_path, "DELETE_NULL", "OGC")
    
    #Intersect WDOECM with basemap
    wdoecm_dissolve_dice_intersect_path = f"path_to_file_geodatabase_for_intermediate_wdoecm_outputs\wdoecm_statusyr{i}_dice2_intersect"
    arcpy.analysis.PairwiseIntersect(
        in_features=[wdoecm_dissolve_dice_path, basemap], 
        out_feature_class=wdoecm_dissolve_dice_intersect_path, 
        join_attributes="ALL", 
        output_type="INPUT")
        
    #Repair geometry
    arcpy.management.RepairGeometry(wdoecm_dissolve_dice_intersect_path, "DELETE_NULL", "OGC")
       
    # Calculate Geometry Attributes for WDOECM
    arcpy.management.CalculateGeometryAttributes(
        wdoecm_dissolve_dice_intersect_path, 
        [["PCA_AREA", "AREA_GEODESIC"]], 
        area_unit="SQUARE_KILOMETERS")
    
    # Save the final outputs
    wdoecm_final_output_path = r"path_to_file_geodatabase_for_final_wdoecm_outputs\WDOECM_Temporal_Coverage_Aug24.gdb"
    wdoecm_dissolved_output_path = os.path.join(wdoecm_final_output_path, f"wdoecm_{month}{year}_statusyr{i}_output")
    arcpy.management.CopyFeatures(wdoecm_dissolve_dice_intersect_path, wdoecm_dissolved_output_path)
    print(f"wdoecm_status_yr_{i}_dissolved saved to {wdoecm_dissolved_output_path}")
    
    #Free ArcGIS Pro memory prior running next iteration.
    arcpy.management.Delete("in_memory")
    arcpy.management.Delete("memory")
    
    end_time = time.time()  # End the timer
    iteration_time = end_time - start_time  # Calculate the time taken for this iteration
    print(f"Iteration {i} took {iteration_time} seconds")
    
print("WDOECM Processing Complete.")

############################################################################################################
### WDPA: Summarise Protected and Conserved Area (PCA_AREA) values by TYPE (land, eez, abnj basemap area)###
############################################################################################################

# Set the workspace to your geodatabase
gdb_path = r"path_to_file_geodatabase_for_final_wdpa_outputs\WDPA_Temporal_Coverage_Aug24.gdb"
arcpy.env.workspace = gdb_path

# Initialize an empty DataFrame to store results
summary_df = pd.DataFrame()

# List all feature classes in the GDB
feature_classes = arcpy.ListFeatureClasses()

# Loop through each WDPA STATUS_YR subset in the geodatabase
for feature_class in feature_classes:
    
    # Create a dictionary to store the summed PCA_AREA values for each type
    area_sums = {}

    # Use an ArcPy SearchCursor to iterate through the features
    with arcpy.da.SearchCursor(feature_class, ["TYPE", "PCA_AREA"]) as cursor:
        for row in cursor:
            feature_type = row[0]
            pca_area = row[1]

            # Skip NULL values in PCA_AREA field
            if pca_area is None:
                continue

            # Add PCA_AREA to the corresponding type's total
            if feature_type in area_sums:
                area_sums[feature_type] += pca_area
            else:
                area_sums[feature_type] = pca_area

    # Convert the area_sums dictionary to a DataFrame for this feature class
    area_df = pd.DataFrame(list(area_sums.items()), columns=["Type", "Total_PCA_Area"])

    # Pivot the DataFrame so each 'Type' becomes a column
    area_df = area_df.pivot_table(index=None, columns="Type", values="Total_PCA_Area").fillna(0)

    # Add a column for the feature class name
    area_df["FeatureClass"] = feature_class

    # Append the current feature class summary to the main DataFrame
    summary_df = pd.concat([summary_df, area_df], axis=0, ignore_index=True)

# Move "FeatureClass" column to the front
cols = summary_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
summary_df = summary_df[cols]

# Save the results to a CSV file
output_csv = r"path_to_file_geodatabase_for_final_wdpa_outputs\WDPA_2000_2024_pca_area_summary.csv"
summary_df.to_csv(output_csv, index=False)

########################################################################################################################
### WDPA (ABNJ only): Summarise Protected and Conserved Area (PCA_AREA) values by TYPE (land, eez, abnj basemap area)###
########################################################################################################################

# Set the workspace to your geodatabase
gdb_path = r"path_to_file_geodatabase_for_final_wdpa_outputs\WDPA_ABNJ_Temporal_Coverage_Aug24.gdb"
arcpy.env.workspace = gdb_path

# Initialize an empty DataFrame to store results
summary_df = pd.DataFrame()

# List all feature classes in the GDB
feature_classes = arcpy.ListFeatureClasses()

# Loop through each WDPA STATUS_YR subset in the geodatabase
for feature_class in feature_classes:
    
    # Create a dictionary to store the summed PCA_AREA values for each type
    area_sums = {}

    # Use an ArcPy SearchCursor to iterate through the features
    with arcpy.da.SearchCursor(feature_class, ["TYPE", "PCA_AREA"]) as cursor:
        for row in cursor:
            feature_type = row[0]
            pca_area = row[1]

            # Skip NULL values in PCA_AREA field
            if pca_area is None:
                continue

            # Add PCA_AREA to the corresponding type's total
            if feature_type in area_sums:
                area_sums[feature_type] += pca_area
            else:
                area_sums[feature_type] = pca_area

    # Convert the area_sums dictionary to a DataFrame for this feature class
    area_df = pd.DataFrame(list(area_sums.items()), columns=["Type", "Total_PCA_Area"])

    # Pivot the DataFrame so each 'Type' becomes a column
    area_df = area_df.pivot_table(index=None, columns="Type", values="Total_PCA_Area").fillna(0)

    # Add a column for the feature class name
    area_df["FeatureClass"] = feature_class

    # Append the current feature class summary to the main DataFrame
    summary_df = pd.concat([summary_df, area_df], axis=0, ignore_index=True)

# Move "FeatureClass" column to the front
cols = summary_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
summary_df = summary_df[cols]

# Save the results to a CSV file
output_csv = r"path_to_file_geodatabase_for_final_wdpa_abnj_outputs\WDPA_ABNJ_2000_2024_pca_area_summary.csv"
summary_df.to_csv(output_csv, index=False)

###############################################################################################################
### WDOECM: Summarise Protected and Conserved Area (PCA_AREA) values by TYPE (land, eez, abnj basemap area)###
###############################################################################################################

# Set the workspace to your geodatabase
gdb_path = r"path_to_file_geodatabase_for_final_wdoecm_outputs\WDOECM_Temporal_Coverage_Aug24.gdb"
arcpy.env.workspace = gdb_path

# Initialize an empty DataFrame to store results
summary_df = pd.DataFrame()

# List all feature classes in the GDB
feature_classes = arcpy.ListFeatureClasses()

# Loop through each WDPA STATUS_YR subset in the geodatabase
for feature_class in feature_classes:
    
    # Create a dictionary to store the summed PCA_AREA values for each type
    area_sums = {}

    # Use an ArcPy SearchCursor to iterate through the features
    with arcpy.da.SearchCursor(feature_class, ["TYPE", "PCA_AREA"]) as cursor:
        for row in cursor:
            feature_type = row[0]
            pca_area = row[1]

            # Skip NULL values in PCA_AREA field
            if pca_area is None:
                continue

            # Add PCA_AREA to the corresponding type's total
            if feature_type in area_sums:
                area_sums[feature_type] += pca_area
            else:
                area_sums[feature_type] = pca_area

    # Convert the area_sums dictionary to a DataFrame for this feature class
    area_df = pd.DataFrame(list(area_sums.items()), columns=["Type", "Total_PCA_Area"])

    # Pivot the DataFrame so each 'Type' becomes a column
    area_df = area_df.pivot_table(index=None, columns="Type", values="Total_PCA_Area").fillna(0)

    # Add a column for the feature class name
    area_df["FeatureClass"] = feature_class

    # Append the current feature class summary to the main DataFrame
    summary_df = pd.concat([summary_df, area_df], axis=0, ignore_index=True)

# Move "FeatureClass" column to the front
cols = summary_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
summary_df = summary_df[cols]

# Save the results to a CSV file
output_csv = r"path_to_file_geodatabase_for_final_wdoecm_outputs\WDOECM_2000_2024_pca_area_summary.csv"
summary_df.to_csv(output_csv, index=False)